Note that the labelling of these sections is based on the version of the paper completed on 15/08/22. This cell should be updated in the event of a change to the labelling in the paper.

Currently running on Sagemath version 9.4.rc0, Release Date: 2021-07-27.
Using Python 3.9.5.

This notebook is intended to be ran sequentially from the beginning. Changing the order in which cells are ran may lead to errors. 

## Isogeny class via Lombardo

In [1]:
# We first write out in Sage some functions that Lombardo will write down in Magma
def ExtractLatticeGenerators(M):
    r"""
    Input a 1xn matrix whose columns span a rank-2 lattice, and reutrn a 1x2
    that spans the same lattice, if possible. If not, return the zero matrix,
    and a logical indicating failure. 
    """
    eps = 1e-4
    v = list(M[0])
    va = [gi.abs() for gi in v]
    n = len(v)
    
    if not any([gi.abs()>eps for gi in v]):
        return matrix.zero(1, 2), False
    g1 = v[va.index(min([gia for gia in va if gia>eps]))]
    
    if not any([gi.abs()>eps and (gi/g1).imag_part().abs()>eps for gi in v]):
        return matrix.zero(1, 2), False
    g2 = v[va.index(min([gi.abs() 
            for gi in v if (gi.abs()>eps and (gi/g1).imag_part().abs()>eps)]))]
    
    CV = matrix.zero(2, n)
    # This loop will do the equivalent of NB's lincomb. 
    for i in range(n):
        CLR = pari.lindep([M[0,i], g1, g2]).sage()
        if not CLR[0]:
            warnings.warn("dependency found betwee g1 and g2 after the fact, this should concern.")
            continue
        CV[0,i] = -CLR[1]/CLR[0]
        CV[1,i] = -CLR[2]/CLR[0]
    L = CV.transpose().LLL()
    B = [r for r in L.rows() if any(r)]
    
    if len(B) != 2:
        warnings.warn("The reduced basis has the wrong length, something has happened")
        
    h1 = B[0][0]*g1 + B[0][1]*g2
    h2 = B[1][0]*g1 + B[1][1]*g2
    
    return matrix([h1, h2]), True

# Because Sage's natural functionality with modular forms is somewhat lacking,
# we will use a modification of the internals of ReconstructEllipticCurve that
# uses NB's theta function and formula from McKean and Moll. 
from riemann_theta.riemann_theta import RiemannTheta

def ReconstructEllipticCurve(ECLattice, K, a):
    InternalC = ComplexField(2000)
    
    if (ECLattice[0][1]/ECLattice[0][0]).imag_part()>0:
        EL = [ECLattice[0][0], ECLattice[0][1]]
    else:
        EL = [ECLattice[0][1], ECLattice[0][0]]
        
    d = K.degree()
    th = RiemannTheta(matrix([EL[1]/EL[0]]))
    
    nw2 = th(char=[[1],[0],2])
    nw3 = th(char=[[0],[0],2])
    nw4 = th(char=[[0],[1],2])
    e1 = S._RR.pi()^2*(nw4^4+nw3^4)/3
    e2 = S._RR.pi()^2*(nw2^4-nw4^4)/3
    e3 = -S._RR.pi()^2*(nw2^4+nw3^4)/3
    g2 = 2*(e1^2 + e2^2 + e3^2)
    g3 = 4*e1*e2*e3
    # These are currently normalaised as if the periods were [1, tau]
    # In fact we know we want the periods to be [EL[0], EL[1]]
    # Hence we need to scale again. 
    g2 *= EL[0]^(-4)
    g3 *= EL[0]^(-6)
    
    if K==QQ:
        g23 = []
        for gi in [g2, g3]:
            deps = pari.lindep([1, gi.real_part()]).sage()
            g23.append(-deps[0]/deps[1])
    else:
        raise NotImplementedError
        
    _.<x> = K[]
    f = 4*x^3-g23[0]*x-g23[1]
    return f

# We can now proceed with the calculation, we initialise our curve
A.<x,y>=AffineSpace(QQ,2)
f = x*(y^5+1) + (x*y)^2 - x^4*y - 2*y^3
C = Curve(f)
S = C.riemann_surface(prec=100)

# Find the endomorphism basis and the a numberf-field approximating the corresponding
# analytic respresentation
E = S.endomorphism_basis()
Aalg = S.tangent_representation_algebraic(E)
K = Aalg[0].base_ring()

# We find Q-endomorphisms as the endormophisms whose analytic respresentation is 
# fixed under the action of the Galois group. This is from BSS+16, equation (6.2.2)
if K != QQ:
    G = K.galois_group()
    g_diffs = [matrix(S.genus, S.genus, [sigma(ai) for ai in Ai.list()]) - Ai 
              for Ai in Aalg for sigma in G.gens()]
    n_matrix = matrix([gi.list() for gi in g_diffs])
    n_Ker = n_matrix.left_kernel()
    n_Ker_mat = n_Ker.matrix()
    E_QQ = [sum([vi*Ei for vi, Ei in zip(v, E)]) for v in n_Ker_mat.rows()]
else:
    E_QQ = E
    
# Once we have found the Q-endomorphisms, we need to find those symmetric under Rosati involution
s_diffs = [S.rosati_involution(Ei) - Ei for Ei in E_QQ]
n_matrix = matrix([gi.list() for gi in s_diffs])
n_Ker = n_matrix.left_kernel()
n_Ker_mat = n_Ker.matrix()

# Finally, once we have imposed these linear conditions, we can impose idempotence via an ideal
# This follows a computation in BSZ19
P = PolynomialRing(QQ, n_Ker_mat.nrows(), names='n')
E_QQ_S = [sum([vi*Ei for vi, Ei in zip(v, E_QQ)]) for v in n_Ker_mat.rows()]

M = sum(a*m for a,m in zip(P.gens(), E_QQ_S))
idempotent_conditions = (M^2-M).list()
idempotent_combinations = [tuple(v[a] for a in P.gens()) 
                           for v in P.ideal(idempotent_conditions).variety()]
idempotents = [sum(a*m for a,m in zip(v, E_QQ_S)) for v in idempotent_combinations]

# We now restrict to the particular class of idemopotents we care about in this case,
# which for now will be ones which correspond to an elliptic subvariety
idempotents_elliptic = [idem for idem in idempotents if idem.rank()==2]
degrees = [lcm([ei.denominator() for ei in idem]) for idem in idempotents_elliptic]
MMs = [deg*idem for deg, idem in zip(degrees, idempotents_elliptic)]

# We can now used the method of Lombardo to reconstruct the elliptic curves. 
P1 = S.period_matrix()

for MM in MMs:
    P2 = P1*MM.change_ring(S._CC)

    for v in GF(2)^S.genus:
        PV = Matrix(v).change_ring(S._CC)
        ELatticeC2 = PV*P2
        ELattice, success = ExtractLatticeGenerators(ELatticeC2)
        if success:
            break
    if not success:
        print("No valid v found")

    f = ReconstructEllipticCurve(ELattice, QQ, 1)

    A.<x,y> = QQ[]
    E = EllipticCurve_from_cubic((y^2-f).homogenize(), morphism=False)
    print(E)
    print(E.j_invariant().factor())
    print(E.label(),"\n")

Elliptic Curve defined by y^2 = x^3 - 145/3*x + 4210/27 over Rational Field
-1 * 2^-5 * 5 * 29^3
50b1 

Elliptic Curve defined by y^2 = x^3 - 25/3*x - 2950/27 over Rational Field
-1 * 2^-1 * 5^2
50a1 



## Relations between induced characters

In [2]:
# We look for 'small' relations between induced characters
S5 = SymmetricGroup(5)
Hs = S5.conjugacy_classes_subgroups()
# We find the matrix with entries corresponding to values on conjugacy class of induced representations
M = matrix([H.trivial_character().induct(S5).values() for H in Hs]).transpose()
M = M.change_ring(QQ)
# Find the kernel of this matrix (which corresponds to relations between induced reps)
KM = M.right_kernel().matrix()
# Perform LLL to look for kernel elements with few non-zero elements
# These correspond to simple relations 
R = KM.LLL()

# We write a function to turn a relation into a string we can print
def reln_to_str(v):
    v = v.list()
    si = min([v.index(ii) for ii in v if ii])
    string = "0 = ({})*(".format(v[si])+Hs[si].structure_description()+")"
    for i in range(si+1, len(v)):
        if not v[i]:
            continue
        string += " + ({})*(".format(v[i])+Hs[i].structure_description()+")"
    return string

# We print relations between only 4 terms
for r in R.rows():
    if sum([ri != 0 for ri in r])==4:
        print(reln_to_str(r))
        
# To get the specifi relations from these we just need to work out which conjugacy class
# is being represented for example when there are two C2 x C2. 

0 = (-1)*(C2) + (1)*(C3) + (1)*(C2 x C2) + (-1)*(A4)
0 = (-1)*(C2) + (1)*(C2) + (-1)*(C2 x C2) + (1)*(C2 x C2)
0 = (-1)*(C2 x C2) + (1)*(C2 x C2) + (-1)*(S3) + (1)*(S3)
0 = (-1)*(C2 x C2) + (1)*(S3) + (1)*(D4) + (-1)*(S4)
0 = (-1)*(C2 x C2) + (1)*(S3) + (1)*(D5) + (-1)*(A5)
0 = (-1)*(S3) + (1)*(D4) + (1)*(A4) + (-1)*(S4)
0 = (-1)*(D4) + (1)*(D6) + (1)*(C5 : C4) + (-1)*(S5)
